# Assignment 3
# CSCI-P556, Spring 2019
## Due date: Monday, March 25, 2019, by 11:59PM.

## Problem 1: Building a Neural Networks using Tensorflow's Low-Level API

The most popular deep learning library is Tensorflow. This library offers two ways develop deep learning models, these are through the High-Level API (Keras) and the Low-Level API. Keras' API is really straight-forward, below we include [Tensorflow's example](https://www.tensorflow.org/tutorials/):

In [119]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

In [120]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


In [82]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [116]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [83]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
60000/60000 [==============================] - 15s 254us/sample - loss: 0.2190 - acc: 0.9349
Epoch 2/5
60000/60000 [==============================] - 15s 258us/sample - loss: 0.0982 - acc: 0.9690
Epoch 3/5
60000/60000 [==============================] - 16s 266us/sample - loss: 0.0677 - acc: 0.9783
Epoch 4/5
60000/60000 [==============================] - 16s 267us/sample - loss: 0.0531 - acc: 0.9831
Epoch 5/5
10000/10000 [==============================] - 1s 67us/sample - loss: 0.0666 - acc: 0.9803


[0.06657557066550653, 0.9803]

Easy, right? Well, we are going to get our hands dirty with the low-level API. **Your task for this problem is to implement a fully-connected deep neural network using [Tensorflow's low-level API](https://www.tensorflow.org/guide).** Make sure that you code can handle passing parameters for the number of layers and hidden units, as well as any other parameter that you deem necessary.

Note: we will take into account your models' accuracy when we grade. For example, if the average accuracy in the class is 91% accuracy and your model is giving 70% accuracy, then you did something wrong. If you aren't getting a good accuracy, check if your loss function is the correct one or try tweaking the hyperparameters.

In [31]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
import tensorflow as tf
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder
from __future__ import print_function
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [16]:
'''
#Used to mount drive in Colab
from google.colab import drive
drive.mount('/content/drive')
'''
### Loading and preprocessing Kuzushiji-49 dataset ###

train_x = np.load(r'C:\Users\vjkri\Desktop\AML\kmnist\k49-train-imgs.npz')['arr_0']
train_y = np.load(r'C:\Users\vjkri\Desktop\AML\kmnist\k49-train-labels.npz')['arr_0']
test_x = np.load(r'C:\Users\vjkri\Desktop\AML\kmnist\k49-test-imgs.npz')['arr_0']
test_y = np.load(r'C:\Users\vjkri\Desktop\AML\kmnist\k49-test-labels.npz')['arr_0']

img_width = train_x[0].shape[0]
img_height = train_x[0].shape[1]

train_x = [train_x[i].flatten() for i in range(len(train_x))]
test_x = [test_x[i].flatten() for i in range(len(test_x))]

#train_x = normalize(train_x, norm = 'l2')
#test_x = normalize(test_x, norm = 'l2')

train_x, test_x = np.array(train_x), np.array(test_x)

train_y = np.reshape(train_y,(train_y.shape[0],1))
test_y = np.reshape(test_y,(test_y.shape[0],1))

#Oversampling minority classses
ros = RandomOverSampler(random_state = 0)
train_x_ros, train_y_ros = ros.fit_resample(train_x, train_y)

C:\Users\vjkri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


232365
294000


In [18]:
#One hot encode for neural nets
op_train, op_test = [], []

for each in train_y_ros:
    temp = np.zeros((49))
    temp[each] = 1
    op_train.append(temp)
for each in test_y:
    temp = np.zeros((49))
    temp[each] = 1
    op_test.append(temp)
    
train_y_nn = np.array(op_train)
test_y_nn = np.array(op_test)

In [24]:
### loading and preprocessing Kuzushiji Kanji dataset ###
import cv2
#give input according to the location of dataset
dir_path = r'C:\Users\vjkri\Desktop\AML\kmnist\kkanji\kkanji2'
dirs = os.listdir(dir_path)
kanji_df = pd.DataFrame(columns = ('X','Y'))

for name in dirs:
    each_class = os.path.join(dir_path,name)
    class_dir = os.listdir(each_class)
    for images in class_dir:
        img_loc = os.path.join(dir_path,name,images)
        img = cv2.imread(img_loc, cv2.IMREAD_GRAYSCALE)
        img = img.flatten()
        kanji_df = kanji_df.append({'X' : img, 'Y' : name}, ignore_index = True)

kanji_df = kanji_df.sample(frac = 1).reset_index(drop=True)

In [25]:
safe_kanji = kanji_df

In [28]:
kanji_x_arr = np.array(list(kanji_df['X']))
kanji_y_arr = np.array(list(kanji_df['Y']))

#Oversampling minority classes 
ros = RandomOverSampler(random_state = 0)
kanji_x_ros, kanji_y_ros = ros.fit_resample(kanji_x_arr, kanji_y_arr)

In [ ]:
#take 20% of dataset as test set
kanji_train_x, kanji_train_y, kanji_test_x, kanji_test_y = train_test_split(kanji_x_ros, kanji_y_ros, test_size = 0.2)

In [ ]:
#one hot encode for neural nets
op_train, op_test = [], []

for each in kanji_train_y:
    temp = np.zeros((3832))
    temp[each] = 1
    op_train.append(temp)
for each in kanji_test_y:
    temp = np.zeros((3832))
    temp[each] = 1
    op_test.append(temp)
    
kanji_train_y_nn = np.array(op_train)
kanji_test_y_nn = np.array(op_test)

In [ ]:
# Your neural network implementation goes here

def neural_net(train_x, train_y, test_x, test_y, learning_rate = 0.01, max_epochs = 30, pixel_dim = 784, classes = 49):
    learning_rate = learning_rate
    max_epochs = max_epochs
    n_hidden_1 = pixel_dim
    n_hidden_2 = 512
    n_hidden_3 = 512
    n_hidden_4 = 512
    n_hidden_5 = 512
    classes = 49
    
    X = tf.placeholder('float', [None, pixel_dim])
    Y = tf.placeholder('float', [None, classes])

    W1 = tf.Variable(tf.random_normal([pixel_dim, n_hidden_1]))
    W2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
    W3 = tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]))
    W4 = tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4]))
    W5 = tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5]))
    Woutput = tf.Variable(tf.random_normal([n_hidden_5, classes]))

    b1 = tf.Variable(tf.zeros([n_hidden_1]))
    b2 = tf.Variable(tf.zeros([n_hidden_2]))
    b3 = tf.Variable(tf.zeros([n_hidden_3]))
    b4 = tf.Variable(tf.zeros([n_hidden_4]))
    b5 = tf.Variable(tf.zeros([n_hidden_5]))
    boutput = tf.Variable(tf.zeros([classes]))

    layer1 = tf.add(tf.matmul(X,W1),b1)
    #layer1 = tf.nn.relu(layer1)
    layer2 = tf.add(tf.matmul(layer1,W2),b2)
     #layer2 = tf.nn.relu(layer2)
    layer3 = tf.add(tf.matmul(layer2,W3),b3)
     #layer3 = tf.nn.relu(layer3)
    layer4 = tf.add(tf.matmul(layer3,W4),b4)
     #layer4 = tf.nn.relu(layer4)
    layer5 = tf.add(tf.matmul(layer4,W5),b5)
    output_layer = tf.add(tf.matmul(layer5,Woutput),boutput)

    logits = output_layer

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))

    optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

    predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))

    accuracy = tf.reduce_mean(tf.cast(predictions, tf.float32))

    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)
        epochs = 1
        while epochs<=max_epochs:
            sess.run(optimize, feed_dict = {X: train_x, Y: train_y})
            loss_ = sess.run(loss, feed_dict = {X: train_x, Y: train_y})
            accuracy_ = sess.run(accuracy, feed_dict = {X: train_x, Y: train_y})
            print('Epoch', epochs)
            #print('Loss', loss_)
            print('Accuracy', accuracy_)
            epochs += 1
            test_acc = sess.run(accuracy, feed_dict = {X: test_x, Y: test_y})
            print('Test accuracy', test_acc)

We will be using two versions of [Kuzushiji dataset](https://arxiv.org/abs/1812.01718) to test your models: Kuzushiji-49 and Kuzushiji-Kanji. The descriptions of datasets are:

> "Kuzushiji-49, as the name suggests, has 49 classes (28x28 grayscale, 270,912 images), is a much larger, but imbalanced dataset containing 48 Hiragana characters and one Hiragana iteration mark."

> "Kuzushiji-Kanji is an imbalanced dataset of total 3832 Kanji characters (64x64 grayscale, 140,426 images), ranging from 1,766 examples to only a single example per class."

If you read the description carefully, you can notice that these aren't datasets that we can just feed to the model without doing some pre-processing. Make sure that your code can handle this issue. Also, the number of layers and hidden units, as well as the hyperparameters are up to you to figure out. 

Instructions on how to download these datasets are available in their [Github page](https://github.com/rois-codh/kmnist). To clone Kuzushiji repository from Github to Google Colab, use:

!git clone https://github.com/rois-codh/kmnist

Note: all Linux commands are available in Google Colab by preceding them with a "!", just like we did above.

In [ ]:
# Test your model using Kuzushiji-49 here
print(neural_net(train_x_ros, train_y_nn, test_x, test_y_nn, learning_rate = 0.01, max_epochs = 100, pixel_dim = 784, classes = 49))

In [ ]:
#Accuracy was 71.3% in Google Colab

In [9]:
# Test your model using Kuzushiji-Kanji here
print(neural_net(kanji_train_x, kanji_train_y, kanj_test_x, kanji_test_y, learning_rate = 0.01, max_epochs = 100, pixel_dim = 4096, classes = 3832))

## Problem 2: Building a Decision Tree using Scikit-Learn's DecisionTreeClassifier

Repeat the above experiments using [Scikit-Learn's DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [ ]:
# Test your model using Kuzushiji-49 here
dt = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 25)
dt = dt.fit(train_x_ros, train_y_ros)
pred = dt.predict(test_x)
print("Accuracy:", accuracy_score(pred, test_y))

In [ ]:
#Accuracy was 54% in Google Colab

In [ ]:
# Test your model using Kuzushiji-Kanji here
dt_kanji = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 25)
dt_kanji = dt_kanji.fit(kanji_train_x, kanji_train_y)
kanji_pred = dt_kanji.predict(kanji_train_x)
print('Accuracy:', accuracy_score(kanji_pred, kanji_test_y))

## Problem 3: Building a Gradient Boosted Tree using Tensorflow's BoostedTreeClassifier estimator

In class we covered the vanilla decision tree. In recent years, a new kind of tree has become very popular in Kaggle competitions due to their performance, these trees are known as [Gradient Boosted Trees](https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting). Typically, the winning submissions for most competitions make use of either some neural network or a gradient boosted tree (either, [XGBoost](https://xgboost.readthedocs.io/en/latest/) or [LightGBM](https://lightgbm.readthedocs.io/en/latest/)). For this problem, we will use [Tensorflow's BoostedTreeClassifier estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/BoostedTreesClassifier). Repeat the above experiments using this model.

In [ ]:
# Test your model using Kuzushiji-49 here
params = {'objective' : 'multiclass', 'num_classes' : 49, 'learning_rate' : 0.001, 'bagging_fraction' : 0.5, 'bagging_freq' : 10}
train_lgb = lgb.Dataset(train_x, label = train_y.ravel())

lgbmodel = lgb.train(params, train_lgb, num_boost_round = 100)
lgb_predict = lgbmodel.predict(train_y)

print('accuracy:', accuracy_score(np.argmax(lgb_predict, axis = 1), test_y))

In [ ]:
#Accuracy was 67.01% in Google Colab with learning rate 0.01

In [ ]:
# Test your model using Kuzushiji-Kanji here
params = {'objective' : 'multiclass', 'num_classes' : 3832, 'learning_rate' : 0.001, 'bagging_fraction' : 0.25, 'bagging_freq' : 5}
train_lgb = lgb.Dataset(kanji_train_x, label = kanji_train_y.ravel())

lgbmodel = lgb.train(params, train_lgb, num_boost_round = 100)
lgb_predict = lgbmodel.predict(kanji_test_x)

print('accuracy:', accuracy_score(np.argmax(lgb_predict, axis = 1), kanji_test_y))